In [94]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split

In [86]:
data = pd.read_csv('data_pivoted.csv')

In [89]:
print data.shape
data = data.fillna(0)
data.head(5)


(65, 317)


,disease,abdomen acute,abdominal tenderness,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,agitation,air fluid level,...,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,acquired immuno-deficiency syndrome,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,adhesion,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,affect labile,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alzheimer's disease,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,anemia,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We need to try both multinomial NB classifier and also binomial classifer. Multinomial classifier is doesn't punish the the non-occurunce of a feature which is a good indicator for a particular disease, whilst Binomial does. 

Not sure which makes more sense in the medical context. 

Need to try both and compare using the right way of doing it (cross validation). Need to read up on evaulating and comparing models. 

Till then will just do a test and train split.

________
###Experiment 1
- Multinomial NB classifier


In [129]:
print data.info()
# data.drop(data.ix[data.symptom.isnull()].index,axis=0, inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65 entries, 0 to 64
Columns: 317 entries, disease to yellow sputum
dtypes: float64(316), object(1)
memory usage: 161.2+ KB
None


In [130]:
cols = data.columns.tolist()
cols.remove('disease')
x = data[cols]
y = data.disease

In [131]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [132]:
mnb = MultinomialNB()
mnb = mnb.fit(x_train, y_train)

In [133]:
mnb.score(x_test, y_test)

0.0

### Inferences on train and test split
It can't work on unseen data because it has never seen that disease before. Also, there is only one point for each disease and hence no point for this. So we need to train the model entirely. Then what will we test it on? Missing data? Say given one symptom what is the disease? This is again multilabel classification. We can work symptom on symptom. What exactly is differential diagnosis, we need to replicate that.

-------
Doing it on the whole set, should the r be 1, and if not, why is it the case, also what r really mean here?


In [134]:
mnb_tot = MultinomialNB()
mnb_tot = mnb_tot.fit(x, y)

In [135]:
mnb_tot.score(x, y)

0.98461538461538467

So the model is misclassifying some diseases, which are those dieases?

In [136]:
disease_pred = mnb_tot.predict(x)

In [137]:
disease_real = y.values

In [138]:
for i in range(0, len(disease_real)):
    if disease_pred[i]!=disease_real[i]:
        print 'Pred: {0} Really:{1}'.format(disease_pred[i], disease_real[i])

Pred: carcinoma prostate Really:malignant neoplasm of prostate


*Inference*: Only one of them is misclassified. TODO: Check why, what are its symtoms.

------
Need a better way of checking the strength of the model. Maybe the paper has some clues to it. 